# Hansel-Spittle with 6 strain rates

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("mime")
import pandas as pd
import h5py
from scipy.optimize import curve_fit
import lmfit

colors = ['#bb0000', '#00bb00', "#0000bb", '#bbbb00', '#bb00bb', "#00bbbb", '#bbbbbb', '#770000', '#007700', "#000077", '#777700', '#770077', "#007777", '#777777', '#440000', '#004400', "#000044", '#444400', '#440044', "#0044444", '#444444','#000000']

In [ ]:
baseSize = (8, 6)  # Base size of a subplot

def sbPlot(n):
    if (n == 1): return 1, 1
    if (n <= 2): return 1, 2
    if (n <= 4): return 2, 2
    if (n <= 6): return 3, 2
    if (n <= 9): return 3, 3
    if (n <= 12): return 4, 3
    return 0, 0

def sbPlotSize(n):
    x, y = sbPlot(n)
    return baseSize[0] * y, baseSize[1] * x

In [ ]:
h5f = h5py.File('../GleebleData.h5','r')
allData = h5f['all'][:]
shortData = h5f['short'][:]
h5f.close()

In [ ]:
allData.shape, shortData.shape

Remove first point of each curve, where $\varepsilon^p=0$

In [ ]:
allData = allData[allData[:,0] != 0]
shortData = shortData[shortData[:,0]!=0]
identData = allData

In [ ]:
strains = np.unique(identData[:,0])
allStrains = np.unique(allData[:,0])
epsps = np.unique(identData[:,1])
temperatures = np.unique(identData[:,2])
nEps = len(strains)
nEpsp = len(epsps)
nTemp = len(temperatures)
strains, epsps, temperatures, nEps, nEpsp, nTemp

# Identification of the Hansel-Spittel parameters
$\sigma^y = Ae^{(m_1T)}\varepsilon^{p^{m_2}}\dot\varepsilon^{p^{m_3}}e^{(\frac{m_4}{\varepsilon^p})}(1+\varepsilon^p)^{(m_5T)}e^{(m_6\varepsilon^p)}\dot\varepsilon^{p^{(m_7T)}} T^{m_8}$

$\ln \sigma^y = \ln A + m_1T + m_2\ln\varepsilon^p + m_3\ln\dot{\varepsilon^p}+ \frac{m_4}{\varepsilon^p} + m_5T\ln (1+\varepsilon^p) + m_6\varepsilon^p + m_7T\ln \dot{\varepsilon^p} + m_8\ln T$

In [ ]:
T0 = temperatures[0]
epsp0 = epsps[0]
Tm = 1460

In [ ]:
def HS5(epsp, depsp, T, opt):
    return opt['A'] * np.exp(opt['m1']*T) * epsp**(opt['m2']) * depsp**(opt['m3']) * np.exp(opt['m4']/epsp) * (1+epsp)**(opt['m5']*T)

def HS6(epsp, depsp, T, opt):
    return opt['A'] * np.exp(opt['m1']*T) * epsp**(opt['m2']) * depsp**(opt['m3']) * np.exp(opt['m4']/epsp) * (1+epsp)**(opt['m5']*T) * np.exp(opt['m6']*epsp)

def HS7(epsp, depsp, T, opt):
    return opt['A'] * np.exp(opt['m1']*T) * epsp**(opt['m2']) * depsp**(opt['m3']) * np.exp(opt['m4']/epsp) * (1+epsp)**(opt['m5']*T) * np.exp(opt['m6']*epsp) * depsp**(opt['m7']*T)

def HS8(epsp, depsp, T, opt):
    return opt['A'] * np.exp(opt['m1']*T) * epsp**(opt['m2']) * depsp**(opt['m3']) * np.exp(opt['m4']/epsp) * (1+epsp)**(opt['m5']*T) * np.exp(opt['m6']*epsp) * depsp**(opt['m7']*T) * T**(opt['m8'])

## 5 Parameters

In [ ]:
param5 = lmfit.Parameters()
param5.add('A', value = 1e3, min = 0.0)
param5.add('m1', value = 0.1)
param5.add('m2', value = 00.1)
param5.add('m3', value = 00.1)
param5.add('m4', value = 00.1)
param5.add('m5', value = 00.1)
param5

In [ ]:
def OF5(opt):
    return identData[:,3] - HS5(identData[:,0], identData[:,1], identData[:,2], opt)
fit5 = lmfit.minimize(OF5, param5)
fit5

## 6 Parameters

In [ ]:
param6 = lmfit.Parameters()
param6.add('A', value = 1e3, min = 0.0)
param6.add('m1', value = 0)
param6.add('m2', value = 0)
param6.add('m3', value = 0)
param6.add('m4', value = 0)
param6.add('m5', value = 0)
param6.add('m6', value = 0)
param6

In [ ]:
def OF6(opt):
    return identData[:,3] - HS6(identData[:,0], identData[:,1], identData[:,2], opt)
fit6 = lmfit.minimize(OF6, param6)
fit6

## 7 Parameters

In [ ]:
param7 = lmfit.Parameters()
param7.add('A', value = 1e3, min = 0.0)
param7.add('m1', value = 0)
param7.add('m2', value = 0)
param7.add('m3', value = 0)
param7.add('m4', value = 0)
param7.add('m5', value = 0)
param7.add('m6', value = 0)
param7.add('m7', value = 0)
param7

In [ ]:
def OF7(opt):
    return identData[:,3] - HS7(identData[:,0], identData[:,1], identData[:,2], opt)
fit7 = lmfit.minimize(OF7, param7)
fit7

## 8 Parameters

In [ ]:
param8 = lmfit.Parameters()
param8.add('A', value = 1e3, min = 0.0)
param8.add('m1', value = 0)
param8.add('m2', value = 0)
param8.add('m3', value = 0)
param8.add('m4', value = 0)
param8.add('m5', value = 0)
param8.add('m6', value = 0)
param8.add('m7', value = 0)
param8.add('m8', value = 0)
param8

In [ ]:
def OF8(opt):
    return identData[:,3] - HS8(identData[:,0], identData[:,1], identData[:,2], opt)
fit8 = lmfit.minimize(OF8, param8)
fit8

In [ ]:
# Plot the curves
from matplotlib.lines import Line2D
def create_dummy_line(**kwds):
    return Line2D([], [], **kwds)

plt.figure(figsize = sbPlotSize(nEpsp))
plt.rc('text', usetex = True)
idx = 1
plt.subplots_adjust(hspace = 0.3)
for epsp in list(epsps):
    xs, ys = sbPlot(nEpsp)
    plt.subplot(xs, ys, idx)
    sbdata = shortData[shortData[:,1] == epsp]
    cl =0
    for temp in list(temperatures):
        sbdata1 = sbdata[sbdata[:,2] == temp]
        plt.plot(sbdata1[:,0], sbdata1[:,3], colors[cl], marker = 's', markersize = 5, linestyle = 'none')
        plt.plot(strains, HS5(strains, epsp, temp, fit5.params), colors[cl], linewidth = 1.5, linestyle='--')
        plt.plot(strains, HS6(strains, epsp, temp, fit6.params), colors[cl], linewidth = 1.5, linestyle='-.')
        plt.plot(strains, HS7(strains, epsp, temp, fit7.params), colors[cl], linewidth = 1.5, linestyle='dashed')
        plt.plot(strains, HS8(strains, epsp, temp, fit8.params), colors[cl], linewidth = 1.5)
        plt.rcParams['xtick.labelsize'] = 16
        plt.rcParams['ytick.labelsize'] = 16
        cl += 1
    plt.xlim(0, 0.7)
    plt.ylim(bottom = 0)
    plt.xlabel(r'strain $\varepsilon$', fontsize = 16) # Labels the x axis
    plt.ylabel(r'flow stress $\sigma^y$ (MPa)', fontsize = 16) # Labels the y axis
    plt.title(r'strain rate $\dot{\varepsilon} = ' + str(epsp) + '$ s$^{-1}$', fontsize = 16) # Self explicit command
    idx += 1

legendLines = []
cl = 0
for temp in list(temperatures):
    legendLines.append((r'$T=$' + str(int(temp)) + r'$^{\circ}$C', {'color':colors[cl], 'linestyle':'-', 'linewidth':2.5, 'marker':'s'}))
    cl += 1

plt.legend([create_dummy_line(**l[1]) for l in legendLines],[l[0] for l in legendLines], 
           loc = 'upper center', fontsize = 12, ncols = 5, bbox_to_anchor = (0.0, -0.2), shadow = False)

plt.savefig("CompExpHS.svg")
plt.show()

In [ ]:
HS5stress = HS5(allData[:,0], allData[:,1], allData[:,2], fit5.params)
HS6stress = HS6(allData[:,0], allData[:,1], allData[:,2], fit6.params)
HS7stress = HS7(allData[:,0], allData[:,1], allData[:,2], fit7.params)
HS8stress = HS8(allData[:,0], allData[:,1], allData[:,2], fit8.params)
HSstress = HS7stress

In [ ]:
data = np.concatenate((allData[:,0:3],HSstress.reshape((HSstress.shape[0],1))),axis=1)
h5f = h5py.File('HS-6.h5','w')
h5f.create_dataset('data', data = data)
h5f.close()

In [ ]:
EAAR5 = np.sum(np.abs((allData[:,3] - HS5stress)/(allData[:,3])))*100/HS5stress.shape[0]
EAAR6 = np.sum(np.abs((allData[:,3] - HS6stress)/(allData[:,3])))*100/HS6stress.shape[0]
EAAR7 = np.sum(np.abs((allData[:,3] - HS7stress)/(allData[:,3])))*100/HS7stress.shape[0]
EAAR8 = np.sum(np.abs((allData[:,3] - HS8stress)/(allData[:,3])))*100/HS8stress.shape[0]
print("EAAR5 = %g" %(EAAR5) + ' %')
print("EAAR6 = %g" %(EAAR6) + ' %')
print("EAAR7 = %g" %(EAAR7) + ' %')
print("EAAR8 = %g" %(EAAR8) + ' %')

In [ ]:
RMSE5 = np.sqrt(np.sum((allData[:,3] - HS5stress)**2)/HS5stress.shape[0])
RMSE6 = np.sqrt(np.sum((allData[:,3] - HS6stress)**2)/HS6stress.shape[0])
RMSE7 = np.sqrt(np.sum((allData[:,3] - HS7stress)**2)/HS7stress.shape[0])
RMSE8 = np.sqrt(np.sum((allData[:,3] - HS8stress)**2)/HS8stress.shape[0])
print('RMSE5 = %g' %(RMSE5)+' MPa')
print('RMSE6 = %g' %(RMSE6)+' MPa')
print('RMSE7 = %g' %(RMSE7)+' MPa')
print('RMSE8 = %g' %(RMSE8)+' MPa')

In [ ]:
def outOf(val, i):
    if ((val<-i) or (val>i)): return True
    return False

def conv (v, d):
    va = abs(v)
    e = int(np.log10(va))
    if (va < 1): e-=1
    if outOf(e,2):
        a = v * 10**(-e)
        v = np.round(a * 10**d)/10**d
        return str(v)+'\\times 10^{'+str(e)+'}'
    v = np.round(v * 10**d)/10**d
    return str(v)

for p in list(fit7.params.values()):
    print(p.name,"=",conv(p.value,4))